In [4]:
# 02_reconciliation.ipynb
# 3-way reconciliation: payments_system vs bank_statement vs refunds
# Outputs: /mnt/data/outputs/reconciliation_summary.csv etc.

import pandas as pd
from pathlib import Path

data_dir = Path('C:/Revenue Leakage')   # change if needed
out_dir = data_dir / 'outputs'
out_dir.mkdir(exist_ok=True)


In [9]:
payments = pd.read_excel(data_dir / 'payments_system.xlsx', parse_dates=['timestamp'])
bank = pd.read_excel(data_dir / 'bank_statement.xlsx', parse_dates=['bank_timestamp'])
refunds = pd.read_excel(data_dir / 'refunds.xlsx', parse_dates=['refund_timestamp'])

print('Loaded rows -> payments:', len(payments), 'bank:', len(bank), 'refunds:', len(refunds))


Loaded rows -> payments: 25000 bank: 18801 refunds: 1224


In [10]:
# standardize types & columns
payments['amount'] = payments['amount'].astype(float)
payments['transaction_id'] = payments['transaction_id'].astype(str)
bank['bank_amount'] = bank['bank_amount'].astype(float)
bank['transaction_id'] = bank['transaction_id'].astype(str)
refunds['refund_amount'] = refunds['refund_amount'].astype(float)
refunds['transaction_id'] = refunds['transaction_id'].astype(str)


In [11]:
p_b = payments.merge(bank, on='transaction_id', how='left', indicator=True)
missing_settlements = p_b[p_b['_merge'] == 'left_only'].copy()
matched_settlements = p_b[p_b['_merge'] == 'both'].copy()

print('Missing settlements (in payments but not in bank):', len(missing_settlements))
print('Matched settlements:', len(matched_settlements))


Missing settlements (in payments but not in bank): 6568
Matched settlements: 18801


In [12]:
# aggregate refunds if multiple refunds per txn
refunds_agg = refunds.groupby('transaction_id').agg({
    'refund_amount': 'sum',
    'refund_timestamp': 'min'   # earliest refund time
}).reset_index()

p_r = payments.merge(refunds_agg, on='transaction_id', how='left', indicator=True)
refunded = p_r[p_r['refund_amount'].notna()].copy()
print('Refunded transactions found in payments dataset:', len(refunded))


Refunded transactions found in payments dataset: 1200


In [13]:
# Over-refunds: refund_amount > amount
refunded['over_refund_flag'] = refunded['refund_amount'] > refunded['amount']

# Refunds for failed transactions
refunded['refund_on_failed'] = (refunded['status'] == 'FAILED') & (refunded['refund_amount'].notna())

# Refunds without matching payments (refund exists but no payments record)
refunds_only = refunds[~refunds['transaction_id'].isin(payments['transaction_id'])].copy()

print('Over-refunds:', refunded['over_refund_flag'].sum())
print('Refunds on FAILED txns:', refunded['refund_on_failed'].sum())
print('Refunds without any payments record (refunds_only):', len(refunds_only))


Over-refunds: 108
Refunds on FAILED txns: 157
Refunds without any payments record (refunds_only): 0


In [14]:
total_overrefund = refunded.loc[refunded['over_refund_flag'], 'refund_amount'].sum() - refunded.loc[refunded['over_refund_flag'], 'amount'].sum()
total_refund_on_failed = refunded.loc[refunded['refund_on_failed'], 'refund_amount'].sum()
total_unmatched_refunds = refunds_only['refund_amount'].sum()

# For missing settlements, compute net difference (system amount - bank amount) for matched rows
miss_settlement_count = len(missing_settlements)
matched_amount_diff = (matched_settlements['amount'] - matched_settlements['bank_amount']).sum()

summary = pd.DataFrame([
    ['Over-Refunds', int(refunded['over_refund_flag'].sum()), float(total_overrefund)],
    ['Refunds on Failed Transactions', int(refunded['refund_on_failed'].sum()), float(total_refund_on_failed)],
    ['Unmatched Refunds', int(len(refunds_only)), float(total_unmatched_refunds)],
    ['Missing Settlements', int(miss_settlement_count), float(matched_amount_diff)]
], columns=['leak_type', 'count', 'amount_lost_kes'])

# Save outputs
summary.to_excel(out_dir / 'reconciliation_summary.xlsx', index=False)
missing_settlements.to_excel(out_dir / 'missing_settlements.xlsx', index=False)
refunded.to_excel(out_dir / 'refunded_detailed.xlsx', index=False)
refunds_only.to_excel(out_dir / 'refunds_unmatched.xlsx', index=False)

summary


,leak_type,count,amount_lost_kes
0,Over-Refunds,108,1.303297e+06
1,Refunds on Failed Transactions,157,3.833359e+06
2,Unmatched Refunds,0,0.000000e+00
3,Missing Settlements,6568,2.900676e+01


In [15]:
# Top customers by refunded amount
top_refunded_customers = refunded.groupby('customer_id').agg({
    'refund_amount': 'sum',
    'amount': 'sum',
    'transaction_id': 'count'
}).rename(columns={'transaction_id':'refund_count'}).sort_values('refund_amount', ascending=False).head(10)

top_refunded_customers.to_csv(out_dir / 'top_refunded_customers.csv')
top_refunded_customers


,refund_amount,amount,refund_count
customer_id,,,
12069,148585.17,82777.09,2
95092,135179.34,46473.70,1
72759,93425.70,93425.70,3
73251,87300.98,43650.49,1
34956,85658.66,42829.33,1
60603,83312.77,83312.77,2
37222,80558.40,40279.20,1
11306,77180.90,38590.45,1
27010,76597.46,38298.73,1


In [16]:
# Example waterfall numbers
total_processed = payments['amount'].sum()
total_refunds = refunds['refund_amount'].sum()
total_missing_settlements = missing_settlements['amount'].sum() if 'amount' in missing_settlements.columns else 0.0
net_collected = total_processed - total_refunds - total_missing_settlements

waterfall = pd.DataFrame([
    ['Total Processed', total_processed],
    ['Total Refunds', total_refunds],
    ['Missing Settlements', total_missing_settlements],
    ['Net Collected', net_collected]
], columns=['metric','kes'])

waterfall.to_csv(out_dir / 'revenue_waterfall.csv', index=False)
waterfall


,metric,kes
0,Total Processed,6.277769e+08
1,Total Refunds,3.071726e+07
2,Missing Settlements,1.639466e+08
3,Net Collected,4.331130e+08
